# BackTrader
BackTrader is a popular Python library for backtesting trading strategies. It is a flexible and extensible framework for creating and testing trading strategies. It is designed to be used by traders, developers, and researchers which can be a powerful tool for backtesting and optimizing trading strategies. 

Backtrader是一个以时间序列数据为基础的回测框架，主要设计用于处理OHLCV（开盘价、最高价、最低价、收盘价和成交量）格式的数据。
在此notebook中包含以下内容：
- Basic Concepts
- Cerebro
- Broker
- Data Feeds
    - 不同数据源的加载 
- Strategies
    - 买卖规则
    - 参数传递
- Indicators
- 可视化结果 

In [26]:
import backtrader as bt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
# matplotlib.use('Qt5Agg')  # 指定使用 Qt5Agg 后端
%matplotlib inline 
# 直接在jupyter notebook中显示图像

## 1. Basic Concepts
### 1. Lines
Lines 表示一系列的数据点，比如价格序列，成交量序列等。对于金融市场数据，一般为：Open, High, Low, Close（OHLC）, Volume等，这些都是时间序列数据。
### 2. Indicators
Indicators 是一种特殊的Line，它是由Lines计算得到的。比如，移动平均线，MACD等。可理解为从基础lines提取的新的特征。
### 3. Index 0
在访问Lines中的值，Index 0表示访问当前值，Index -1表示访问上一个值，Index 1表示访问下一个值。
由此，若想在初始化期间创建一个简单移动平均线的策略，可以这样写：
```python
class MyStrategy(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(self.data)
# 访问此移动平均线的当前值，可以这样写：
av=self.sma[0]
# 访问此移动平均线的上一个值，可以这样写：
av_previous=self.sma[-1]
``` 

## 2. Cerebro
`Cerebro` 是Backtrader的心脏，是整个回测和交易系统的控制中心。你可以把它想象成大脑（Cerebro来自于西班牙语单词“大脑”），负责协调各个部分的工作，包括数据加载、策略执行、订单生成、记录和可视化等。使用Cerebro的基本步骤包括：

1. **实例化Cerebro**：首先需要创建一个Cerebro对象来开始构建你的回测系统。`cerebro = bt.Cerebro()`。
2. **添加策略**：通过`addstrategy()`方法向Cerebro中添加交易策略。 `cerebro.addstrategy(MyStrategy)`。
3. **加载数据**：将市场数据加载到Cerebro中，这些数据可以是CSV文件、Pandas DataFrame等。 `cerebro.adddata(data)`。
4. **设置初始资金**：可以通过`broker.setcash()`方法设置策略的初始资本。 `cerebro.broker.setcash(100000.0)`。
    - `getcash` 方法用于查询当前的现金余额。随着策略的执行，买卖操作会影响现金余额。可以让你了解在任何给定时间点，策略账户中剩余的现金量。
5. **执行回测**：通过调用`run()`方法来执行策略的回测。 `cerebro.run()`。
6. **结果分析和可视化**：回测完成后，可以通过`plot()`方法可视化回测结果，也可以通过Cerebro的其他方法进行性能分析。`cerebro.plot()`。

## 3. Broker

`Broker` 在Backtrader中代表经纪人，负责订单的执行和资金的管理。在实际交易中，经纪人是执行买卖订单的实体，而在Backtrader中，Broker对象模拟了这一行为。主要功能包括：

1. **执行订单**：根据策略生成的买卖信号执行订单。 
2. **资金管理**：管理策略的资金，包括初始资金、当前现金余额、持仓价值等。 `broker.setcash()`、`broker.getcash()`。
3. **设置佣金**：可以通过`setcommission()`方法设置交易佣金，以模拟真实交易环境中的成本。 `broker.setcommission(commission=0.001)`。
4. **记录交易**：记录每一笔交易的详细信息，如交易时间、价格、数量等。`broker.notify_order()`、`broker.notify_trade()`。

在Cerebro中，默认会创建一个Broker实例，但你也可以自定义Broker的行为，比如自定义滑点模型、佣金结构等，以更加贴近实际交易环境。
通过这两个组件，Backtrader为用户提供了一个强大且灵活的框架，不仅可以进行历史数据的策略回测，还可以对策略进行优化，并最终应用于实时交易。

## 4. Data Feeds
在Backtrader中添加数据是进行回测的重要步骤之一，因为数据提供了策略运行所需要的市场信息。Backtrader支持多种数据格式，包括CSV文件、Pandas DataFrame、在线数据源等。下面将介绍几种常见的添加数据的方法及相应的代码示例。

#### 1. 从CSV文件添加数据

假设你有一个CSV文件，里面包含了股票的历史价格数据，格式可能如下：

```
Date,Open,High,Low,Close,Volume,OpenInterest
2020-01-01,100,110,95,105,1000,0
...
```

你可以使用`bt.feeds.YahooFinanceCSVData`（或`bt.feeds.GenericCSVData`，如果你的CSV格式与Yahoo财经的格式不完全一致）来加载这个CSV文件：

```python
import backtrader as bt

# 创建Cerebro引擎对象
cerebro = bt.Cerebro()

# 加载CSV数据
data = bt.feeds.YahooFinanceCSVData(
    dataname='path_to_your_csv_file.csv',
    fromdate=datetime.datetime(2020, 1, 1),
    todate=datetime.datetime(2021, 1, 1)
)

# 将数据添加到Cerebro
cerebro.adddata(data)
```

Yahoo财经提供的股票市场数据通常包含以下字段：

1. **Date**：交易日期，通常格式为YYYY-MM-DD。
2. **Open**：开盘价，即该交易日开始时的价格。
3. **High**：最高价，该交易日内的最高交易价格。
4. **Low**：最低价，该交易日内的最低交易价格。
5. **Close**：收盘价，即该交易日结束时的价格。
6. **Adj Close**：调整后的收盘价，考虑了股票分红、派息、股票分割等因素后的收盘价。
7. **Volume**：成交量，表示在该交易日内完成交易的股票数量。

Yahoo财经的CSV数据文件通常会包含这些列，如下所示的示例格式：

```
Date,Open,High,Low,Close,Adj Close,Volume
2020-01-02,74.059998,75.150002,73.797501,75.087502,74.333511,135480400
2020-01-03,74.287498,75.144997,74.125000,74.357498,73.610840,146322800
...
```

### 使用 GenericCSVData

当使用Backtrader的`GenericCSVData`类来加载非标准格式的CSV数据时，你可以通过指定参数来匹配你的数据列。`GenericCSVData`类提供了灵活性来定义每列数据的名称、位置以及格式。如果你的CSV文件格式与Yahoo财经的标准格式不完全一致，你可以调整`GenericCSVData`的参数以适应你的数据，如下所示：

```python
import backtrader as bt

# 自定义CSV数据格式
data = bt.feeds.GenericCSVData(
    dataname='your_data_file.csv',
    
    fromdate=datetime.datetime(2020, 1, 1),
    todate=datetime.datetime(2021, 1, 1),
    
    nullvalue=0.0,
    
    dtformat=('%Y-%m-%d'),  # 日期格式
    datetime=0,  # 日期所在列
    open=1,  # 开盘价所在列
    high=2,  # 最高价所在列
    low=3,  # 最低价所在列
    close=4,  # 收盘价所在列
    volume=5,  # 成交量所在列
    openinterest=-1,  # 如果数据中没有未平仓合约量，可以设置为-1
    # 如果你的数据中包含调整后的收盘价，也可以添加相应的参数
    adjclose=6,  # 调整后收盘价所在列（如果有的话）
)
```

在这个例子中，你需要根据你的CSV文件的实际格式调整列的位置（`datetime=0, open=1, high=2, ...`等），以及日期格式（`dtformat=('%Y-%m-%d')`）。如果你的CSV文件中某些数据列不存在（例如，没有`OpenInterest`），可以将对应的参数设置为`-1`。

通过正确设置`GenericCSVData`中的参数，你可以使Backtrader适应几乎任何格式的CSV数据文件，进行有效的策略回测。

#### 2. 使用Pandas DataFrame添加数据

如果你的数据以Pandas DataFrame的形式存在，可以使用`bt.feeds.PandasData`来添加数据。首先，确保你的DataFrame至少包含以下列：'datetime', 'open', 'high', 'low', 'close', 'volume', 'openinterest'。其中'datetime'列应为Pandas的DatetimeIndex。

```python
import backtrader as bt
import pandas as pd

# 假设df是你的Pandas DataFrame
# df = pd.read_csv('path_to_your_csv_file.csv')
# df['datetime'] = pd.to_datetime(df['Date'])
# df = df.set_index('datetime')

# 创建Cerebro引擎对象
cerebro = bt.Cerebro()

# 使用Pandas DataFrame加载数据
data = bt.feeds.PandasData(dataname=df)

# 将数据添加到Cerebro
cerebro.adddata(data)
```

####  3. 从在线数据源添加数据

Backtrader也支持从在线数据源直接加载数据，例如从Yahoo Finance。使用`bt.feeds.YahooFinanceData`或`bt.feeds.YahooFinanceAPI`（取决于你想如何访问数据）可以直接从Yahoo Finance获取数据。

```python
import backtrader as bt

# 创建Cerebro引擎对象
cerebro = bt.Cerebro()

# 从Yahoo Finance加载数据
data = bt.feeds.YahooFinanceData(
    dataname='AAPL',
    fromdate=datetime.datetime(2020, 1, 1),
    todate=datetime.datetime(2021, 1, 1)
)

# 将数据添加到Cerebro
cerebro.adddata(data)
```

在添加数据时，你可以根据需要自定义数据源的参数，比如时间范围、时间框架（日线、分钟线等）、数据调整（复权处理）等。

---

Cerebro的索引方式：
- `cerebro.datas[0]`：获取第一个数据源；若同时feed了多个数据源，可以通过`cerebro.datas[1]`、`cerebro.datas[2]`等来获取其他数据源。
- `cerebro.datas[0].close`：获取第一个数据源的收盘价。
    - `cerebro.datas[0].close[0]`：获取第一个数据源的当前收盘价;若为`cerebro.datas[0].close[-1]`，则获取上一个收盘价。
- `cerebro.datas[0].datetime`：获取第一个数据源的日期时间。
- `cerebro.datas[0].volume`：获取第一个数据源的成交量。

## 5. Strategies
策略是Backtrader中的核心组件，它定义了交易逻辑和执行规则。在Backtrader中，策略是一个Python类，继承自`bt.Strategy`。你可以在策略中定义买卖信号、止损规则、止盈规则等。下面是一个简单的策略示例：

```python
class MyStrategy(bt.Strategy):
    def __init__(self):
        # 添加移动平均线指标
        self.sma = bt.indicators.SimpleMovingAverage(self.data)

    def next(self):
        # 如果当前价格高于移动平均线，买入
        if self.data.close[0] > self.sma[0]:
            self.buy()

        # 如果当前价格低于移动平均线，卖出
        elif self.data.close[0] < self.sma[0]:
            self.sell()
```

## 6. 代码示例
下面将使用从Yahoo Finance下载的苹果公司（AAPL）股票数据(2022-01-03--->2024-02-20)，来演示如何使用Backtrader进行简单的策略回测。首先，我们需要加载数据，然后定义一个简单的策略，最后执行回测并可视化结果。该数据是一个csv文件，格式为标准Yahoo Finance格式，包含了股票的历史价格数据（Open, High, Low, Close, Volume）。

In [59]:
#获取数据,使用YahooFinanceData（从官网下载的Apple股票数据）
df = pd.read_csv("E:\\Bristol\\mini_project\\JPMorgan_Set01\\AAPL_ohlc.csv",index_col='Date',parse_dates=True)
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,177.830002,182.880005,177.710007,182.009995,179.724548,104487900
2022-01-04,182.630005,182.940002,179.119995,179.699997,177.443558,99310400
2022-01-05,179.610001,180.169998,174.639999,174.919998,172.723587,94537600
2022-01-06,172.699997,175.300003,171.639999,172.000000,169.840240,96904000
2022-01-07,172.889999,174.139999,171.029999,172.169998,170.008118,86709100


### 1. 创建Cerebro引擎对象
通过`bt.Cerebro()`创建一个Cerebro对象，它是整个回测和交易系统的控制中心。设置希望观测的时间段，然后加载数据。
下面这段代码一般用于脚本中的主程序部分，即`if __name__ == '__main__':`之后的部分

### 2. 定义策略
目前已经有了金额和股票信息数据，接下来需要自定义策略来让Cerebro执行。投资策略，针对的通常是数据的收盘价（close），也就是根据收盘价决定如何投资。下面是一个简单的策略示意，该策略仅仅打印出收盘价。
这段代码定义了一个名为`TestStrategy`的简单Backtrader策略类。下面是代码的逐行解释：

### 类定义
```python
class TestStrategy(bt.Strategy):
```
这行代码定义了一个新的策略类`TestStrategy`，它继承自`bt.Strategy`。在Backtrader中，所有的交易策略都需要继承自`bt.Strategy`类。

### 日志记录功能
```python
def log(self, txt, dt=None):
    """ 提供记录功能"""
    dt = dt or self.datas[0].datetime.date(0)
    print('%s, %s' % (dt.isoformat(), txt))
```
这个`log`方法是一个辅助函数，用于记录（打印）策略执行过程中的重要信息。它接受一个字符串`txt`和一个可选的日期`dt`。如果没有提供`dt`，它将使用当前处理的数据条目的日期。这个方法简化了日志记录过程，使得在策略的其他部分可以轻松地记录信息。

### 策略初始化
```python
def __init__(self):
    # 引用到输入数据的close价格
    self.dataclose = self.datas[0].close
```
在策略的`__init__`方法中，进行了初始化设置。这里，它创建了一个对数据集中第一个数据（`self.datas[0]`）的收盘价（`close`）的引用。这意味着你可以在策略的其他部分通过`self.dataclose`来访问当前的收盘价。

### next方法
```python
def next(self):
    # 目前的策略就是简单显示下收盘价。
    self.log('Close, %.2f' % self.dataclose[0])
```
`next`方法是策略的核心，它在每个数据点（例如，每个交易日）被调用一次。在这个简单的策略中，`next`方法仅使用之前定义的`log`方法来打印当前数据点的收盘价。`self.dataclose[0]`是当前时间点的收盘价，而`self.dataclose[-1]`将是上一个时间点的收盘价。

strategy 的next方法针对self.dataclose（也就是收盘价Line）的每一行（也就是Bar）进行处理。在本例中，只是打印了下close的值。next方法是Strategy最重要的的方法，具体策略的实现都在这个函数中，后续还会详细介绍。

### 总结
总的来说，这个策略非常基础，它仅仅在每个交易日打印当日的收盘价。尽管简单，但它展示了使用Backtrader创建策略的基本结构，包括日志记录、初始化设置以及如何在每个时间点处理数据。这个策略可以作为更复杂策略的起点，通过添加买卖逻辑、指标计算等来进一步发展。

### 3. 添加交易规则
对Strategy类进行扩展，添加交易规则。在这个例子中，我们将添加一些买卖操作
- 买入规则：当价格连续两天下跌时买入
- 卖出规则：持有5天后卖出。本例中使用5个bar后卖出，而bar本身不涉及时间，由于这里数据采样频率是天，所以每个bar恰好代表一天。

这个完整的Backtrader脚本定义了一个名为`TestStrategy`的交易策略类，并使用了一些基本的Backtrader功能来进行交易模拟和回测。下面是对类中各个方法的解释：

### log 方法
```python
def log(self, txt, dt=None):
```
这个方法用于记录（打印输出）策略执行过程中的信息。它接受一个文本消息`txt`和一个可选的日期参数`dt`。如果没有提供`dt`，它会使用当前处理的数据点的日期。这个日志方法便于策略开发者跟踪策略的执行情况和重要事件。

### __init__ 方法
```python
def __init__(self):
```
这是类的构造函数，用于策略的初始化。在这里，它做了三件事：
1. 保持对第一个数据系列（`datas[0]`）中收盘价（`close`）的引用，以便在策略中使用。
2. 初始化`self.order`为`None`，用于跟踪当前是否有挂起的订单。
3. 初始化`self.buyprice`和`self.buycomm`为`None`，用于记录最后一次买入操作的价格和佣金。

### notify_order 方法
```python
def notify_order(self, order):
```
这个方法在每个订单状态发生变化时被Backtrader调用（例如，订单被接受、完成或取消）。它用于处理订单状态的变化，如记录订单完成时的价格和佣金，以及在订单被取消或拒绝时输出相关信息。此外，当订单完成后，它会重置`self.order`为`None`，表示当前没有挂起的订单。

### notify_trade 方法
```python
def notify_trade(self, trade):
```
这个方法在每个交易完成时被Backtrader调用。一个“交易”指的是一个完整的买入并随后卖出的过程。这个方法用于记录交易的盈利情况，包括毛利润（`trade.pnl`）和净利润（考虑佣金后的利润，`trade.pnlcomm`）。

### next 方法
```python
def next(self):
```
这是策略的核心方法，每当新的数据点到来时被调用一次。在这个方法中，策略会检查当前是否有挂起的订单、是否持有仓位，然后根据定义的买入卖出规则执行交易。在这个示例中，买入规则是如果发现连续两天收盘价下跌，就创建买入订单；卖出规则是在买入后的第五天创建卖出订单。

### 设置初始资金和佣金
```python
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)
```
这两行设置了策略的初始资金为100,000单位（假设是美元），并设置了每笔交易的佣金率为0.1%。

### 开始和结束时的投资组合价值
```python
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
...
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
```
这两行分别在策略运行前后打印投资组合的初始和最终价值，以评估策略的表现。

通过结合这些方法和设置，`TestStrategy`类实现了一个简单的交易策略，它在检测到特定价格行为时执行买卖操作，并通过Backtrader的框架进行模拟和回测。

### 4. 参数传递
在Backtrader中，可以通过在策略类的`__init__`方法中定义参数，然后在添加策略到Cerebro引擎时传递这些参数，来给策略传递参数。这允许你在不修改策略代码的情况下测试不同的参数值，从而进行策略优化和灵活的回测。

#### 1. 定义策略参数

首先，在你的策略类中定义参数。这通常是通过在类级别使用`params`属性来完成的，如下所示：

```python
class TestStrategy(bt.Strategy):
    params = (
        ('param1', 10),
        ('param2', 20),
    )

    def __init__(self):
        # 使用策略参数
        self.param1 = self.params.param1
        self.param2 = self.params.param2
```

这里，我们定义了两个参数`param1`和`param2`，并分别赋予了它们默认值10和20。

#### 2. 传递参数给策略

在将策略添加到Cerebro引擎时，可以通过`addstrategy`方法传递参数值。如果你不传递某个参数，它将使用在策略定义中设置的默认值。

```python
# 创建Cerebro引擎实例
cerebro = bt.Cerebro()

# 向Cerebro引擎添加策略，并传递参数
cerebro.addstrategy(TestStrategy, param1=15, param2=25)

# 在nest()方法中使用策略参数
def next(self):
    # 使用策略参数，假设设定为在持有日期为param1后卖出
    if len(self) >= (self.bar_executed + self.param1):
        self.log('SELL CREATE, %.2f' % self.dataclose[0])
        self.order = self.sell()
```

在这个例子中，我们将`param1`的值设为15，`param2`的值设为25。这些值将在策略的`__init__`方法中被接收并设置。

#### 3. 使用优化器传递参数

如果你想要进行参数优化，可以使用`optstrategy`方法而不是`addstrategy`。`optstrategy`允许你为参数提供一个值列表，Backtrader将会遍历这些值来执行策略优化。

```python
# 使用optstrategy进行参数优化
cerebro.optstrategy(TestStrategy, param1=range(10, 20), param2=range(20, 30))
```

在这个例子中，`param1`将从10遍历到19，`param2`将从20遍历到29，Backtrader将为这些参数值的每种组合运行策略，让你能够找到最优的参数组合。

通过这种方式，你可以灵活地给策略传递不同的参数，测试和优化你的交易策略。

## 5. Indicators
Indicators 是一种特殊的Line，它是由Lines计算得到的。比如，移动平均线，MACD等。可理解为从基础lines提取的新的特征。

#### 创建指标
在定义策略时，可在`__init__`方法中创建指标。Backtrader提供了许多内置的指标，如简单移动平均线（SMA）、指数移动平均线（EMA）、相对强弱指标（RSI）等。你可以使用这些内置指标，也可以自定义指标。

```python
class TestStrategy(bt.Strategy):
    def __init__(self):
        # 创建一个简单移动平均线指标
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=15)
        # 创建一个指数移动平均线指标,该指标没有用self指定，即不会用于策略逻辑，只用于绘图
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25, subplot=False)
```


Backtrader使用matplotlib库来生成策略运行后的图表，其中会包括价格图、指标线以及交易点等信息。每个指标都会根据其所属的数据系列（例如价格数据）和配置（如是否在子图中显示）来适当地放置在图表中。

如果你想控制指标的作图行为，比如是否在主图中显示或是单独在子图中显示，可以在指标创建时使用`subplot`参数。例如，如果你不希望某个指标显示在主图中，可以设置`subplot=True`：

```python
self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod, subplot=True)
```

这会将SMA指标放在单独的子图中，而不是和价格数据一起在主图中显示。反之，如果`subplot=False`（默认值），则指标将与价格数据一起显示在主图中。

需要注意的是，即使指标在策略逻辑中被广泛使用，其在图表中的显示也完全取决于指标的作图配置和Backtrader的作图系统。如果在某些情况下你不希望某个指标显示在图表中，可以通过设置指标的`plot`参数为`False`来实现：

```python
self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod, plot=False)
```

这样，即使指标被用于交易逻辑，它也不会出现在最终生成的图表中。

## 6. 可视化结果
在Backtrader中，绘图是通过内置的绘图功能实现的，该功能基于Matplotlib库。默认情况下，当你运行策略后调用`cerebro.plot()`方法时，Backtrader会生成并显示策略的性能图表。下面是如何在Backtrader中进行绘图的基本步骤：

#### 在策略中添加指标
在你的策略类中定义并添加你想要在图表中显示的指标。你已经在策略中添加了多个指标，如SMA、EMA等。

#### 运行策略 并绘图
正常运行你的策略，通常是设置初始资金，添加数据源，添加策略类，然后调用`cerebro.run()`：

```python
cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy)
cerebro.adddata(data)
cerebro.broker.setcash(10000.0)
cerebro.run()
cerebro.plot()
```

#### 可选: 自定义绘图
`plot()`方法接受多个参数，允许你自定义图表的某些方面。例如：

- `style`：绘图风格，如`'bar'`、`'candle'`、`'line'`等。
- `barup`：上涨K线的颜色。
- `bardown`：下跌K线的颜色。
- `volume`：是否绘制成交量图。

一个自定义绘图的例子：

```python
cerebro.plot(style='candlestick', barup='green', bardown='red', volume=True)
```

### 注意事项
- 在某些环境中（如某些IDE或Jupyter Notebook），Backtrader的绘图功能可能无法正常工作。如果遇到问题，尝试在标准Python脚本中运行你的代码，或者检查你的环境配置。
- 如果你的数据量非常大，绘图可能会变得缓慢或消耗较多资源。在这种情况下，考虑减少数据量或调整绘图参数以优化性能。

通过这些步骤，你可以在Backtrader中绘制出包含价格数据、指标和交易点等信息的图表，有助于直观地分析策略的性能和市场行为。

保存图表为文件：
`cerebro.plot(plotdir='/path/to/save/plots', style='candlestick')`


In [16]:
# 定义策略
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15), # 移动平均线窗口大小
        ('rsiperiod', 14),  # RSI指标的周期
        ('rsiupper', 70),  # RSI过高阈值，超过此值可能考虑卖出
        ('rsilower', 30),  # RSI过低阈值，低于此值可能考虑买入
    )
    
    # 日志记录
    # 若传入了dt参数，则使用传入的日期，否则使用数据本身的日期
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
 
    def __init__(self):
        # 保持对收盘价的引用，即只关注收盘价这一列并用于策略
        self.dataclose = self.datas[0].close
 
        # 初始化order（订单状态），buyprice（买入价格）和buycomm（买入佣金）
        self.order = None
        self.buyprice = None
        self.buycomm = None
 
        # 用于买卖策略的指标
        # sma: 简单移动平均线
        # rsi: 相对强弱指标
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)
        self.rsi = bt.indicators.RSI(self.datas[0], period=self.params.rsiperiod)

        # 仅用于绘图的指标
        # subplot=True表示在原图上叠加显示，否则单独显示
        # plot=False表示不用于作图，但是可以用于策略逻辑
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25, subplot=False)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25, subplot=True)
        bt.indicators.StochasticSlow(self.datas[0], subplot=True)
        bt.indicators.MACDHisto(self.datas[0], subplot=True)
        bt.indicators.ATR(self.datas[0], plot=False)
    
    # 每个订单状态发生变化时被Backtrader调用（例如，订单被接受、完成或取消）。它用于处理订单状态的变化，如记录订单完成时的价格和佣金，以及在订单被取消或拒绝时输出相关信息。
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
 
        # 若order状态为Completed，则进行下一步判断
        if order.status in [order.Completed]:
            # 订单已经完成，若为买入，则记录买入价格price，总花费cost（value），佣金comm
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
                # 更新买入价格和佣金
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            
            # 若为卖出，则记录卖出价格price，总收入value，佣金comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
 
            self.bar_executed = len(self)
        
        # 若order状态为Canceled/Margin/Rejected，则记录订单取消/保证金不足/拒绝
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
 
        # 重置订单状态
        self.order = None
    
    # 一个trade是一次完整的买入和卖出过程，该函数用于记录交易的盈利情况
    def notify_trade(self, trade):
        # 若trade不是空且trade不是已经关闭 则退出该函数
        if not trade.isclosed:
            return
        # 若trade已经关闭，则记录交易的毛利润pnl和净利润pnlcomm
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
    # 核心策略执行函数
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
        
        # 若有订单正在进行中，则不进行任何操作
        if self.order:
            return
        
        # 若没有持有仓位，则进行买入判断
        # 策略逻辑：当价格高于SMA且RSI低于上限时买入；当价格低于SMA且RSI高于下限时卖出
        if not self.position:
            if self.dataclose[0] > self.sma[0] and self.rsi[0] < self.params.rsiupper:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
        else:
            if self.dataclose[0] < self.sma[0] and self.rsi[0] > self.params.rsilower:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

In [30]:
# 创建Cerebro引擎对象
cerebro = bt.Cerebro()
start_date = datetime(2022, 1, 3)  # 回测开始时间
end_date = datetime(2024, 2, 20)  # 回测结束时间

# 加载数据
data = bt.feeds.YahooFinanceData(dataname="E:\\Bristol\\mini_project\\JPMorgan_Set01\\AAPL_ohlc.csv", fromdate=start_date, todate=end_date)

# 将数据传入回测系统
cerebro.adddata(data)  
# 设置初始资金和佣金
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
# 添加该策略到Cerebro
cerebro.addstrategy(TestStrategy)
# 可以使用optstrategy进行参数优化代替addstrategy
# cerebro.optstrategy(
#     TestStrategy,
#     param1=range(10, 20),  # param1将从10遍历到19
#     param2=[20, 30, 40]    # param2将在这三个值中进行遍历
# )
# 增加每次交易的股票数量
cerebro.addsizer(bt.sizers.FixedSize, stake=70)
# 运行回测
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# 绘制图表
%matplotlib inline
cerebro.plot(figuersize=(1000,500))

Starting Portfolio Value: 100000.00
2022-02-18, Close, 165.41
2022-02-22, Close, 162.46
2022-02-23, Close, 158.26
2022-02-24, Close, 160.90
2022-02-25, Close, 162.99
2022-02-28, Close, 163.25
2022-03-01, Close, 161.36
2022-03-02, Close, 164.68
2022-03-03, Close, 164.35
2022-03-04, Close, 161.33
2022-03-07, Close, 157.50
2022-03-08, Close, 155.66
2022-03-09, Close, 161.11
2022-03-10, Close, 156.73
2022-03-11, Close, 152.98
2022-03-14, Close, 148.92
2022-03-15, Close, 153.34
2022-03-16, Close, 157.79
2022-03-17, Close, 158.81
2022-03-17, BUY CREATE, 158.81
2022-03-18, BUY EXECUTED, Price: 158.70, Cost: 11109.00, Comm 11.11
2022-03-18, Close, 162.13
2022-03-21, Close, 163.51
2022-03-22, Close, 166.91
2022-03-23, Close, 168.29
2022-03-24, Close, 172.10
2022-03-25, Close, 172.75
2022-03-28, Close, 173.62
2022-03-29, Close, 176.94
2022-03-30, Close, 175.76
2022-03-31, Close, 172.64
2022-04-01, Close, 172.34
2022-04-04, Close, 176.42
2022-04-05, Close, 173.08
2022-04-06, Close, 169.89
2022-04

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 8 Axes>]]